In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d
import corner
from orbit_class import Orbit
import misc_utils as mu   
import slit_prism_noise_model as spn
import simulate_data as simdata 

In [ ]:
# R = 50
# Band 2: 660 nm 15 % width 
# Band 3: 730 nm 15 % width
# Band 1: 575 nm 10 % width
# Band 4: 825 nm 10 % width
c2,l2,r2 = 0.66, 0.66*(1-0.075),0.66*(1+0.075)
c3,l3,r3 = 0.73, 0.73*(1-0.075),0.73*(1+0.075)
c1,l1,r1 = 0.575, 0.575*(1-0.05),0.575*(1+0.05)
c4,l4,r4 = 0.825, 0.825*(1-0.05),0.825*(1+0.05)

def R2(wl):
    x1,y1 = 0.610, 77.5
    x2,y2 = 0.710, 46.0
    slope = (y1-y2)/(x1-x2)
    intercept = y2-x2*slope
    return intercept + slope*wl

def R3(wl):
    x1,y1 = 0.675, 76.0
    x2,y2 = 0.785, 47.0
    slope = (y1-y2)/(x1-x2)
    intercept = y2-x2*slope
    return intercept + slope*wl

###### CORONAGRAPH PARAMETERS SPC REQ + SLIT PRISM
# TELESCOPE
D = 2.37                        
tau_obs = 0.835                  
Apm = np.pi*(D/2.0)**2.0*tau_obs   
# DETECTOR 
eta = quantum_efficiency      
cte_cr_hot = 0.596*0.838*0.95   # (REQ goes with 63 months at L2)
phc = 0.90                     
tfr =  80.0                   
qcic = 0.0232                  
idark =  5.56*10.0**-4.0            
sig_read = 0.00000001          
# CORONAGRAPH
cperformance_table = np.loadtxt("AuxiliaryData/newSPC.dat",unpack=True)
tablesampling = 0.2 
tau_ref = 0.337 
OWA = 2.8 
IWA = 8.6 
# IFS
wl_c = 0.66 # where the det is nyquist sampled
mpix = 53.0 
R = 50.0
# POST-PROCESSING
fpp = 1.0/12.0

req_spc_dict = {'Apm':Apm,'eta':eta,'tfr':tfr,'qcic':qcic,'idark':idark,'phc':phc,'cte_cr_hot':cte_cr_hot,'R':R,
      'sig_read':sig_read,'wl_c':wl_c,'fpp':fpp,'OWA':OWA,'IWA':IWA,'D':D,'mpix':mpix,'tau_obs':tau_obs,'tau_ref':tau_ref}

req_spc_pars = ast_constants_dict.copy() 
update_c_version(req_spc_dict, req_spc_pars, cperformance_table, tablesampling)

In [ ]:
params = {}                    # create a dictionary for your simulation's parameters
params.update({'d_obs':10.0})  # distance to the observer in parsecs
params.update({'Mstar':5.0})   # absolute stellar V-band magnitude
params.update({'Nez':1.0})     # exozodi level
stellartype = 'g0v'            # stellar type
stellar_mass = 1.0             # stellar mass in units of solar mass 
                               # this is only used in orbit object, not really relevant for SNR calculations
stellar_spec = 'AuxiliaryData/'+stellartype+'.dat' # imports Hubble 
ref_wl, ref_flambda = np.loadtxt(stellar_spec, unpack=True, usecols=(0,1))
fstar_lambda = interp1d(ref_wl,ref_flambda)  # specific flux density W / m^2 / micron, for zero mag star
params.update({'fstar_lambda':fstar_lambda}) # a function which returns 
                                             # specific flux density for any wavelength
params.update({'rp':1.0})      # planet radius in Jupiter radii
params.update({'chromo':0.75,'meth':0.25}) # short wavelength (chromophore) and long wavelength (methane)
                                           # weightings for linear combination of Jupiter and Neptune 
params.update({'Ag':hpm.juneper_Agfunc(params['chromo'],params['meth'])}) 
a = 3.8            # semimajor axis (in au)
ecc = 10.0**-8.0   # eccentricity
inc = 90.0         # inclination (degrees)
ome = 0.0         # longitude of ascending node (degrees)
tp = 0.0           # epoch of perihelion passage (julian date)
argperi = 0.0     # argument of perihelion (degrees)
orbit_pars = np.array([ecc,inc,ome,tp,a,argperi,
                       stellar_mass,params['rp'],params['d_obs']])
orbit = Orbit(orbit_pars) 